In [2]:
from RNN2 import LSTM
import torch
import numpy as np 
from torch import nn
from sklearn.preprocessing import MinMaxScaler
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibili

In [2]:
################
#  DATALOADING #
################

In [4]:

def sliding_windows(data,seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[(i+seq_length):(i+1 + seq_length)]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)



In [5]:
bsz = 200

data = np.load('latent-action.npy')
#Reshaping data and seperating training and test set
sc = MinMaxScaler()
data = sc.fit_transform(data.squeeze(axis = 0))

seq_length = 4
x, y = sliding_windows(data, seq_length)



train_size = int(len(y) * 0.67)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])).float())
trainY = Variable(torch.Tensor(np.array(y[0:train_size])).float())


testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])).float())
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])).float())


trainingset =[]
for i in range(len(trainY)):
    trainingset.append((trainX[i,:,:], trainY[i]))
    
testset =[]
for i in range(len(testY)):
    testset.append((testX[i,:,:], testY[i]))
    

trainloader = torch.utils.data.DataLoader(trainingset, batch_size=bsz, shuffle=False)
testloader = torch.utils.data.DataLoader(testset,batch_size = 1)
print(trainloader)




In [29]:

learning_rate = 1e-7
input_size = 18
hidden_size = 256
num_layers = 1
output_size = 18 


mdnrnn = LSTM(output_size, input_size, hidden_size, num_layers, seq_length, fc1_out)


def detach(states):
    return [state.detach() for state in states]

In [35]:
optimizer = torch.optim.Adam(mdnrnn.parameters(), lr = learning_rate)

epochs = 500
random = torch.zeros(200,4,17)

for epoch in range(epochs):
    # Set initial hidden and cell states
    hidden = mdnrnn.init_hidden(bsz)

    # Get mini-batch inputs and targets
    total_loss = 0
    j = 0
    for i, (inputs, targets) in enumerate(trainloader):
        # Forward pass
        if inputs.size(0) != bsz:
            continue
        hidden = detach(hidden)
        (pi, mu, sigma), hidden = mdnrnn(inputs.to(device), hidden)
        loss = mdnrnn.criterion(targets.to(device), pi, mu, sigma)
        # Backward and optimize
        mdnrnn.zero_grad()
        total_loss += loss.item()

        loss.backward()
        clip_grad_norm_(mdnrnn.parameters(), 0.5)
        optimizer.step()
        j += 1
    total_loss /=j
    if epoch % 2 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch, epochs, total_loss))
        torch.save(mdnrnn.state_dict(), '../Models/transition-regression-target')




Epoch [0/500], Loss: 1.0586
Epoch [2/500], Loss: 1.0412
Epoch [4/500], Loss: 1.0188
Epoch [6/500], Loss: 0.9858
Epoch [8/500], Loss: 0.9291
Epoch [10/500], Loss: 0.7946
Epoch [12/500], Loss: 0.4079
Epoch [14/500], Loss: 0.1890
Epoch [16/500], Loss: 0.0048
Epoch [18/500], Loss: -0.1460


KeyboardInterrupt: 

In [14]:
learning_rate = 1e-3
input_size = 18
hidden_size = 256
num_layers = 1
num_classes = 1
fc1_out = 64
output_size = 18

lstm = LSTM(output_size, input_size, hidden_size, num_layers, seq_length, fc1_out)
lstm.load_state_dict(torch.load('../Models/transition-gpu-0.001', map_location='cpu'))

lstm.eval()
zero = np.random.randint(testX.size(0))

one = np.random.randint(testX.size(1))

x = testX[zero:zero+1, one:one+1, :]
y = testX[zero:zero+1, one+1:one+2, :]

prediction = lstm(x).detach().numpy()
print(prediction.shape)

prediction = sc.inverse_transform(prediction)
truth = sc.inverse_transform(y.squeeze(0))
print(prediction)
print(truth)

(1, 18)
[[ 3.3912663   0.7882177   2.6937652  -1.4916643  -0.80562866  2.7565274
   0.5399529  -2.490175    0.5358663   0.47130808  0.43136594  0.5235378
   0.57759076  0.50562733  0.3721605   0.1992499   0.02955814  1.034743  ]]
[[ 3.08009347e+00  7.79383769e-01  2.42883431e+00 -2.15578349e+00
  -5.63320990e-01  2.29998605e+00 -5.88748919e-02 -2.18521358e+00
   5.03000025e-01  4.99999982e-01  4.32999991e-01  5.27999985e-01
   5.44000024e-01  4.94000000e-01  3.85000004e-01  1.54999997e-01
  -1.13248825e-08  0.00000000e+00]]


In [ ]:
torch.save(mdnrnn.state_dict(), '../Models/transition-regression-new2')

In [ ]:
import pickle
with open('sc', 'wb') as f:
    pickle.dump(sc, f)
    